In [147]:
pip install pyspark

In [148]:
from pyspark.sql import SparkSession
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import IntegerType, StringType, StructType, StructField, LongType


In [149]:
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark


In [150]:
spark = init_spark()


In [151]:
# read the data file with userID, itemId and session time
dataDf = spark.read.option("delimiter", ",").option("header", True).csv("../data/data_v1.csv/part-00000-a0a21325-e951-4449-a031-1860d3dfd170-c000.csv")

In [152]:
# ! temporary
# remove the extra visit number from the user id
# this should be done in main
# we need integer user ids for the ALS algorithm

dataRdd = rddFromFile.rdd.map(lambda x: (int(x[0].split('-')[0]), x[1], int(x[2])))

In [153]:
# a few userIds are longer than long integers and aren't parsing properly ( this is only a few users)

filteredRdd = dataRdd.filter(lambda x: x[0] < 9223372036854775807)

In [154]:
# split the training and testing data

mappedDf = spark.createDataFrame(filteredRdd, schema)
(training, test) = mappedDf.randomSplit([0.8, 0.2])
training.show()

+-----------------+--------------+----------------+
|           userID|        itemID|session_duration|
+-----------------+--------------+----------------+
| 8016723867009901|GGOEACCQ017299|               0|
| 8016723867009901|GGOEGBJL013999|            1419|
| 8016723867009901|GGOEGBPB021199|            1292|
| 8016723867009901|GGOEGCKQ013199|               1|
| 8016723867009901|GGOEGDHC074099|            1394|
| 8016723867009901|GGOEGESB015099|            1488|
| 8016723867009901|GGOEGESC014099|            1322|
| 8974926326637231|  GGOEGAAX0325|           13752|
| 8974926326637231|  GGOEGAAX0351|           16293|
|11577526465501235|  GGOEGAAX0037|               1|
|11577526465501235|  GGOEGAAX0341|               1|
|11577526465501235|GGOEGCNB021099|            1951|
|11577526465501235|GGOEGOLC013299|               0|
|11577526465501235|GGOEGPJC203399|               1|
|24507252193437459|GGOEGFKQ020799|           11192|
|34988253365419131|GGOEGBRB013899|           11630|
|34988253365

In [146]:
# try to train the dataset how the assignment did it

# item id must be numeric apparently:/ todo
als = ALS(maxIter=5, regParam=0.01, userCol="userID", itemCol="itemID", ratingCol="session_duration", coldStartStrategy="drop")
model = als.fit(training)

IllegalArgumentException: requirement failed: Column itemID must be of type numeric but was actually of type string.